In [2]:
from layers import *
import tensorflow as tf

In [5]:
from layers import *
import tensorflow as tf

bs=64
field_num=23
emb_size=16
inputs=tf.ones((bs,field_num,emb_size))
MLP(out_dim=3,hidden_units=[128,64])(inputs)
SELayer([128])(inputs)

## Dinattention
query=tf.ones((bs,emb_size))
key=tf.ones((bs,10,emb_size))
seqlen=tf.ones((bs))*5
DinAttention()(key,query,seqlen)

## ppnet
inputs=tf.ones((bs,1288))
gate_emb=tf.ones((bs,128))
PPNetLayer(out_dim=64,hidden_units=[128],gate_hidden_units=[128,128])(inputs,gate_emb)

## fibi
inputs=tf.ones((64,23,16))
BilinearInteraction()(inputs)

## cin
inputs=tf.ones((64,23,16))
CinLayer([128,128])(inputs)

## Fm
inputs=tf.ones((64,23,16))
FMCrossLayer()(inputs)

## MMoE
inputs=tf.ones((bs,1288))
MMoELayer(4,[256],2,[128])(inputs)

## PLE
inputs=tf.ones((bs,1288))
PLELayer(1,4,[256],2,[128],1)(inputs)


[<tf.Tensor: shape=(64, 256), dtype=float32, numpy=
 array([[0.09439056, 0.4682368 , 0.14988433, ..., 0.5949478 , 0.5036044 ,
         0.57740164],
        [0.09439056, 0.4682368 , 0.14988433, ..., 0.5949478 , 0.5036044 ,
         0.57740164],
        [0.09439056, 0.4682368 , 0.14988433, ..., 0.5949478 , 0.5036044 ,
         0.57740164],
        ...,
        [0.09439056, 0.4682368 , 0.14988433, ..., 0.5949478 , 0.5036044 ,
         0.57740164],
        [0.09439044, 0.46823692, 0.14988431, ..., 0.5949477 , 0.5036043 ,
         0.57740164],
        [0.09439044, 0.46823692, 0.14988431, ..., 0.5949477 , 0.5036043 ,
         0.57740164]], dtype=float32)>,
 <tf.Tensor: shape=(64, 256), dtype=float32, numpy=
 array([[0.06685781, 0.483452  , 0.1261303 , ..., 1.3969997 , 0.4793982 ,
         0.34164172],
        [0.06685781, 0.483452  , 0.1261303 , ..., 1.3969997 , 0.4793982 ,
         0.34164172],
        [0.06685781, 0.483452  , 0.1261303 , ..., 1.3969997 , 0.4793982 ,
         0.34164172],
 

In [63]:
a=tf.ones((64,12))
b=tf.ones((64,12))


In [67]:
inputs=tf.ones((64,23,16))
obj=BilinearInteraction()
obj(inputs)

253 23


<tf.Tensor: shape=(64, 253, 16), dtype=float32, numpy=
array([[[-1.4013947 , -1.7496046 ,  0.16214529, ..., -0.00360952,
          1.4629452 , -0.9206601 ],
        [ 0.63977605, -0.08518679, -1.070574  , ..., -0.27574095,
          1.4065473 , -0.88943446],
        [ 1.4094085 ,  0.5798226 , -0.78099006, ..., -1.602323  ,
         -0.9113116 ,  0.8146272 ],
        ...,
        [-1.7533984 ,  0.5607278 , -1.5570295 , ...,  0.4228723 ,
         -0.5006953 , -0.37363836],
        [-0.33253673,  0.80279505, -0.00845532, ...,  2.1672962 ,
          0.6889852 ,  0.852971  ],
        [ 0.274296  , -1.5495188 , -0.55503577, ..., -1.2353386 ,
          0.00488108,  0.5857283 ]],

       [[-1.4013947 , -1.7496046 ,  0.16214529, ..., -0.00360952,
          1.4629452 , -0.9206601 ],
        [ 0.63977605, -0.08518679, -1.070574  , ..., -0.27574095,
          1.4065473 , -0.88943446],
        [ 1.4094085 ,  0.5798226 , -0.78099006, ..., -1.602323  ,
         -0.9113116 ,  0.8146272 ],
        ...,

In [20]:
a=tf.ones((64,12,16))
b=tf.concat([tf.ones((1,16,16))*i for i in range(12)],axis=0)
ans=tf.einsum("ijk,jkk->ijk",a,b)

In [21]:
ans.shape

TensorShape([64, 12, 16])

In [30]:
import itertools
n=3
for i,j in itertools.combinations(range(n), 2):
    print(i,j)

0 1
0 2
1 2


In [7]:
tf.nn.rnn_cell

AttributeError: module 'tensorflow._api.v2.nn' has no attribute 'rnn_cell'

In [6]:
import itertools

import tensorflow as tf
from tensorflow.keras import layers

class DynamicMultiRNN(layers.Layer):
    def __init__(self, num_units=None, rnn_type='LSTM', return_sequence=True, num_layers=2, num_residual_layers=1, dropout_rate=0.2,
                 forget_bias=1.0, **kwargs):

        self.num_units = num_units
        self.return_sequence = return_sequence
        self.rnn_type = rnn_type
        self.num_layers = num_layers
        self.num_residual_layers = num_residual_layers
        self.dropout = dropout_rate
        self.forget_bias = forget_bias
        super(DynamicMultiRNN, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        input_seq_shape = input_shape[0]
        if self.num_units is None:
            self.num_units = input_seq_shape.as_list()[-1]
        if self.rnn_type == "LSTM":
            single_cell = tf.nn.rnn_cell.BasicLSTMCell(self.num_units, forget_bias=self.forget_bias)
        elif self.rnn_type == "GRU":
            single_cell = tf.nn.rnn_cell.GRUCell(self.num_units)
        else:
            raise ValueError("Unknown unit type %s!" % self.rnn_type)
        dropout = self.dropout if tf.keras.backend.learning_phase() == 1 else 0
        single_cell = tf.nn.rnn_cell.DropoutWrapper(cell=single_cell, input_keep_prob=(1.0 - dropout))
        cell_list = []
        for i in range(self.num_layers):
            residual = (i >= self.num_layers - self.num_residual_layers)
            if residual:
                single_cell_residual = tf.nn.rnn_cell.ResidualWrapper(single_cell)
                cell_list.append(single_cell_residual)
            else:
                cell_list.append(single_cell)
        if len(cell_list) == 1:
            self.final_cell = cell_list[0]
        else:
            self.final_cell = tf.nn.rnn_cell.MultiRNNCell(cell_list)
        super(DynamicMultiRNN, self).build(input_shape)

    def call(self, input_list, mask=None, training=None):
        rnn_input, sequence_length = input_list
        with tf.name_scope("rnn"), tf.variable_scope("rnn", reuse=tf.AUTO_REUSE):
            rnn_output, hidden_state = tf.nn.dynamic_rnn(self.final_cell, inputs=rnn_input, sequence_length=tf.squeeze(sequence_length),
                                                         dtype=tf.float32, scope=self.name)
        if self.return_sequence:
            return rnn_output
        else:
            return tf.expand_dims(hidden_state, axis=1)

    def compute_output_shape(self, input_shape):
        rnn_input_shape = input_shape[0]
        if self.return_sequence:
            return rnn_input_shape
        else:
            return (None, 1, rnn_input_shape[2])

    def get_config(self, ):
        config = {'num_units': self.num_units, 'rnn_type': self.rnn_type, 'return_sequence':self.return_sequence, 'num_layers': self.num_layers,
                  'num_residual_layers': self.num_residual_layers, 'dropout_rate': self.dropout}
        base_config = super(DynamicMultiRNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))